## 특징점
> 특징점이란 '영상에서 특징이 될 만한 지점', '영상에서 중요 정보를 가지고 있다고 판단되는 지점'이다.

- 특징점을 통해서 각기 다른 이미지, 영상간의 대응 관계를 수립할 수 있다.
- 그렇다면, 특징점 Matching의 세부 과정을 살펴보자.

**특징점 Matching 세부과정**
1. Detection - 검출 : 독특한 특징점들을 검출함
2. Description - 기술 : 특징점 주위의 픽셀들이 어떤 모양을 가지는지 정량적인 벡터로 도출 = feature descripter
    - ex) 특징점을 기준으로 한 local window의 주변 픽셀들의 밝기 값을 비교했을 때 차이가 적으면 같음을 인식할 수 있다. 이렇게, 서로 다른 영상의 '특징점이 서로 유사한지 비교할 수 있는 데이터'가 descripter인 것이다! </br></br>
3. Matching - 매칭 : 특징 벡터간의 거리를 계산하여 대응 관계를 수립한다 -> 거리가 짧은 특징점 벡터들끼리 정합을 하면 특징점간의 대응 관계가 수립이 된다!
***

**좋은 특징점 및 특징기술자의 특성**
1. 구별가능성 Distinctiveness </br>
: 같거나 다름을 구별하기 위해서 독특한 특징을 지니고 있어야 한다. 그러한 구별가능성을 갖고 있는 점을 특징점으로 설정한다.

2. 반복가능성 Repeatability </br>
: 두 영상에서 동일한 지점이 어느 정도 이상 특징적으로 검출되게 하고자 한다.

3. 소형화 Compactness & 효율성 Efficiency</br> 
: 검출된 특징점을 중심으로 local window를 잡고 그 윈도우의 픽셀의 분포에 대한 벡터 표현 즉, 기술자를 생성할때 그 기술자 자체가 최대한 작은 것이 좋다.</br>
->why?) 특징점들을 기술자를 통해 정합을 할 때, 여러가지 경우의 수를 고려해야 하므로 계산의 효율성에 있어 기술자의 크기가 작을수록 좋다! + 데이터 사이즈 측면에서 저장에도 용이함

***

### 특징점 검출과정

- 도출된 특징점이 상대 영상의 어느 픽셀 위치와 대응되는지는 미리 알 수 없다.
- 그러나 특징점으로 뽑은 픽셀에서의 작은 위치 변화에 따라 생김새appearance가 얼마나 안정적인지 아닌지를 계산할 수 있음.
    - 특징점을 중심으로하는 local window의 픽셀의 변화가 크다 -> 중요한 정보로 생각할 수 있음.
- 영상의 패치를 주변 이웃과 비교함

**Corner Detection**
- 작은 윈도우 내부에서 모서리Corner를 인식함.
- 어느 방향으로 이동하든 픽셀 밝기값의 intensity분포가 크게 변화되는 window를 찾고자 함.

![2](img/2.png)

1. Flat region : 모든 방향으로 변화 없음
2. Edge : 가장자리 방향으로 변화 없음
3. Corner : 모든 방향으로 변화함 -> 특징점이 될 수 있다! </br>
-> why?) 어느 방향으로 움직이든 변화가 크다는 것은 그 점이 독특한 특성을 갖고 있다는 것이고, 이에 따라 그 점의 위치가 드러나기 때문이다. 그러므로 '구별가능성'이 생기는 것이다! </br>
-> 즉, Corner point가 keypoint = feature point가 될 수 있다! </br>

그렇다면, Corner점을 어떻게 찾을 수 있을까?

***

**Auto-correlation에 의한 코너 검출**
- 자기 자신과 주변 영역간의 상관관계를 계산하는 것을 의미한다

![3](img/3.png)

- 위의 그림과 수식을 보면, keypoint가 될 수 있는 특정한 점을 기준으로 한 window를 u, v만큼 옮기고 겹치는 부분의 픽셀간의 차의 제곱의 합을 구하는 것이다.
- w(x, y) : Window function / I(x+u, y+v) : Shifted intensity / I(x, y) : intensity
- 위의 값이 높을수록 변화가 크다고 생각하면 된다.
- 즉, 작은 변화에 대해 E(u, v)값이 어떻게 변화되는지 관찰하고자 하는 것이다. 그러나, 이 계산을 전부 수행하는 데에는 아주 많은 시간이 소요된다.
    - O(window_width^2 * shift_range^2 * image_width^2) </br>

![4](img/4.png)
![5](img/5.png)

- 위의 그림처럼 강한 봉우리를 갖는 지점을 찾고자 하는 것이다!
- 그러므로, E(u, v)의 좁은 지역을 국소적으로 2차 표면으로 근사하는 방법을 택한다! -> '테일러 급수 전개'활용

![6](img/6.png)

- (0, 0)근방에서 E(u, v)의 2계도 근사값은 2차 테일러 급수 전개에 의해 도출된다.
- 위의 수식에서 E(0, 0)은 f(0, 0)의 값, [u, v]는 (x - (0,0))을 의미, 2x1행렬은 각각 u, v에 대한 편미분값, 두번째 항에서 2x2행렬은 u, v에 대한 2계도 편미분값을 의미하고 [u, v]와 전치행렬의 곱은 xT*x를 의미한다. 즉, 2차 테일러 급수식에 해당하는 것이다!

![7](img/7.png)
![8](img/8.png)
![9](img/9.png)

- 이렇게 얻은 (0, 0)근방에서 E(u, v)의 2계도 근사식에서 우리는 얼마나 봉우리가 깊은지, 즉 2차함수가 얼마나 가파른지에 대한 정보를 얻고 싶은 것이다!
- 그렇다면, 2차함수의 가파름은 2계도 미분값에 의해 결정되므로, 앞의 항들은 0으로 설정하고 2계도 미분값만 확인하도록 한다.






![10](img/10.png)

- M은 결국 2계도함수를 이용하여 만들어진 Hessian행렬이다!

***Hessian Matrix***
- Hessian행렬은 어떤 함수의 2계 도함수들을 이용하여 행렬을 만든 것이다.
- 헤시안 행렬의 원소는 모두 이계도함수로 이루어져 있는데, f의 이계도함수가 연속이라면 헤세 행렬은 '대칭 행렬'이다.

- <2차 미분계수의 의미>
- 2차 함수 f(x) = (1/2a)*x^2 + bx + c 가 있다고 하자. 이 함수 f(x)의 이계도함수는 a인데, a가 양수라면 아래로 볼록, 음수라면 위로 볼록한 함수 형태를 갖는다.
- a의 크기가 클수록 이차함수의 폭이 좁아져 더 볼록한 형태를 갖는다는 특징 또한 알 수 있다!

***Hessian Matrix의 기하학적 의미***
- 모든 행렬은 선형변환으로 대체해 생각할 수 있다. 그리고, 선형 변환을 기하하적으로 생각하면 선형 변환이란 일종의 공간 변형이라고도 생각할 수 있다.
- 기하학적으로 헤시안 행렬이 시행하고 있는 선형 변환은 기본 bowl형태의 함수를 좀 더 볼록하거나 오목하게 만드는 변환이다!
- 그렇다면 헤시안 행렬이 보여주는 기하학적인 변환의 주요한 특징을 파악하기 위해서는 어떻게 해야될까? -> 선형 변환의 중심축을 찾고, 그 크기를 수치화해야한다!
- 그러므로 헤시안 행렬의 고유값과 고유벡터를 파악함으로써 가능할 것이다!!!

cf) 선형 대수 개념
- M은 정사각 대칭 행렬이다.
- 대칭 행렬의 고유벡터들은 모두 서로 직교한다.
- 정사각 행렬 M은 M = P^-1AP를 만족시키는 임의의 행렬 P가 존재하면 대각화 할 수 있다.
- 이때 A는 대각행렬이며, P는 기저변화를 시키는 행렬이다.(2D에서는 P가 회전행렬이 된다.)

![11](img/11.png)

- 위와 같이 M행렬을 대각화하여 고유값에 의해 타원 축들의 길이가 결정되는 것을 알 수 있다.
- 방향을 나타내는 R행렬은 우리에게 중요치 않다! -> 우리는 Cornerness만 따지면 되므로, 방향은 의미가 없고 그러므로 M행렬을 대각화하여 얻은 고유값에 의해 Cornerness를 판단하면 된다!
- 더 큰 고유값일수록 기울기가 가파르고, 시각화할 때 더 작은 타원으로 나타내진다.

![12](img/12.png)
![13](img/13.png)

- 위의 그림과 수식을 살펴보자.
- 두 고유값이 둘 다 클때만 C값이 양수를 갖게끔 Cornerness Score를 정의한 것이다.
- 수식을 살펴보면, 고유값의 곱은 M이 대칭행렬이므로 M의 행렬식으로 표현할 수 있고, 정사각 대칭 행렬의 trace는 고유값의 합이므로 'C = det(M) - α * trace(M)^2'로 표현할 수 있다!! 결과는 밑의 수식과 같다!

![14](img/14.png)





***
**Harris corner detector**
- 각 윈도우에 대한 M행렬을 계산하여 cornerness score = C를 도출한다!
    - cf) 픽셀 당 이미지 미분값에서 M을 찾을 수 있다. </br>
- Cornerness score C중에 큰 값을 찾아내기 위해 임계값을 정하고 임계값 미만의 값은 걸러냄(C > threshold)
- 주변 픽셀 대비 국소적으로 C값이 최대인 local maximum 픽셀들을 선별한다.

***
**Harris corner점의 Invariance & Covariance**
- 해리스 코너 점의 위치가
    - 1) 광도photometric 변화에 불변invariant하고,
    - 2) 기하하적geometric 변화에 공변covariant하는가? </br>
- Invariance : 영상이 변환되고 난 후에도 코너의 위치는 변경되지 않음.
- Covariance : 동일한 장면의 변환된 두 영상에 대해, 코너점이 대응되는 동일한 위치에 있음.

> Affine intensity change의 경우
- 미분값derivatives만 미용됨 -> 밝기값 평행변황에 대해 불변하다. : I -> I + b
- 밝기값 스케일링 intensity scaling : I -> aI

> Image translation : 이미지 자체의 평행이동인 경우
- 영상 미분과 윈도우 함수는 평행이동에 불변하다.

> Image rotation : 이미지의 회전
- 2차 모멘트 행렬의 타원은 회전하지만 고유 값 등 형태는 동일하게 유지된다.

<font color=Blue><결론적으로, 해리스 코너 점은 광도와 기하하적 변화에 각각 불변, 공변하다!></font>

But, 코너의 위치는 스케일링Scaling에 대해 공변하지 않다!!! </br>

![15](img/15.png)

- 위의 그림처럼 원래 코너점이었던 것이 스케일이 스케일 변화에 따라 edge로 분류되어버린다!

***
**특징점에서 'SCALE'이란 무엇인가?**

![16](img/16.png)

- 위처럼 같은 영상이지만 하나는 조금 더 멀리서, 하나는 가까운 곳에 찍은 영상이 있다고 하자.
- 왼쪽의 작은 패치(window)와 오른쪽의 조금 더 큰 패치에 대하여 같은 함수값을 도출하게 하는 적절한 패치 크기를 어떻게 찾고, 이 때의 함수는 어떻게 정의하면 될까?
- 패치의 크기를 변화시키며 함수에 대한 응답을 찾아가보자.

![17](img/17.png)

- 패치의 크기를 점점 증가시키면서 함수 f에 대한 응답을 살펴보면 위와 같다.
- 패치 크기의 증가에 따른 함수 f에 대한 응답(scale signature) 즉, 스케일 별 함수 응답의 어떤 특징적인 분포를 통해서 이 영상의 위치가 특징점이 된다면 그 때의 적절한 스케일이 얼마일지를 구할 수 있는 것이다!
- scale signature에서의 local maximmum point를 찾아서 그것을 스케일로 도출한다는 아이디어를 기반으로, 적절한 크기별 scale signature을 도출시켜주는 함수 'f'를 찾으면 될것이다!!

***

**적절한 scale signature을 도출해주는 적절한 함수 f는 무엇일까?**

- 특징적인 위치를 정의하는데 일반적으로 "덩어리blob" 검출기를 많이 활용한다!
    - Laplacian(2nd derivative) of Gaussian(LoG) </br>
- LoG 커널을 가지고 영상을 컨볼루션하면, blob부분에서 반응성이 높게 나온다.
- 또한, LoG 커널의 사이즈를 다르게하면 다른 scale의 blob을 검출해낼 수 있다.

![18](img/18.png)

- 위의 그림처럼 자신 주변의 픽셀에서 뿐만 아니라, 인접한 스케일에 대해서도 반응성이 큰 극댓값을 갖는 지점을 찾는 것이다!
- 즉, Position-scale 공간에서 국소적 최대(극대)값을 찾으면 그 지점이 특징점의 후보가 되는 것이다! -> (x, y, s) : 좌표값 뿐만 아니라, scale값도 도출해낼 수 있다.

***

**Alternative Kernel**

> 그러나, LoG를 통한 필터링 연산을 반복하면 굉장히 비효율적이다. 그에 비해서 단순하게 Gaussian filter를 통한 연산은 훨씬 효율적이다. 그러므로, LoG를 Gaussian filter를 통해서 근사하여 연산을 처리하면 효율적일 것이다.

![19](img/19.png)
![20](img/20.png)

- Difference-of-Gaussian(DoG)를 통해 LoG를 근사하여 계산하자!
- σ Gaussian kernel로 영상 블러링 한 것과 kσ Gaussian kernel로 블러링 한 결과를 빼줌으로써 LoG를 근사한 값을 계산해낼 수 있다.

![21](img/21.png)

- 위의 그림은 DoG연산을 통해서 구한 결과의 이미지이다.
- 원의 크기는 scale을 의미하고, 원 안의 십자가는 scale signature에 대하여 극대값인 점을 의미한다.
- 이제 우리는 반응 크기(scale response)가 극대인 점에 대해 기술자를 생성해야 한다!



***
### 기술(Description)

***지역적 영상 기술자 Local Image Descriptor***
> 각 특징점 주위의 특징 기술자를 벡터로 추출해보자.

![22](img/22.png)

- 영상의 특성을 표현하는 기법인 히스토그램Histogram이 있다.
- 전역적 히스토그램은 feature의 분포를 나타내준다.
- 우리가 위에서 검출한 특징점 주변의 지역적 히스토그램은 어떤 의미를 가질까?
- 그리고 어떤 것들에 대해 히스토그램을 계산하게 될까?
    - 무늬(Texture) : 특징점 주변의 edge성분들의 분포
    - edge 성분들의 그래디언트 방향성의 지역적 히스토그램 -> "지역적 기술자"를 만들 수 있을 것이다! =>> <font color=Red>SIFT : Scale Invariant Feature Transform</font>





***
**SIFT Preprocessing**
1. 특징점의 위치로 DoG scale-space 최대값 찾기

2. 특징점 검출 결과 후처리 Post-processing
    - Subpixel 위치 보간 : 픽셀 밝기값의 소수점 단위의 보간 적용 -> 특징점의 위치를 소수점 단위로 정밀하게 보정
    - 대비가 낮은 points 제거
    - 가장자리를 따라 points 제거 : 가우시안 필터링은 영상의 가장자리에서는 결과가 잘 도출될 수 없기 때문이다. </br></br>

3. 특징점당 방향 추정
    - 그래디언트 방향 히스토그램 계산
    - 지배적인dominant 방향 선택 -> θ : 특징점에 (x, y, s, θ) 기준 방향까지 할당할 수 있음. </br></br>

4. 기술자 추출
    - 크기와 방향이 주어진 특징점 :
        - 추정된 스케일과 가장 가깝세 대응하는 scale-space 영상을 선택
        - 방향에 맞게 영상을 다시 샘플링 하거나
        - 임의의 영상 회전에 대해 불변하도록 <font color=Blue>방향 벡터들로부터 검출된 특징점의 지배적인 방향을 빼서 상대적인 방향 특성을 기술화한다.</font> -> 이를 방향 정규화 과정이라고 한다.

![23](img/23.png)

- 위의 그림처럼 4x4 즉, 16개 픽셀에 대하여 각 그래디언트의 크기(=진폭)를 합산하여 8방향 '히스토그램'으로 나타낸다. -> gradient orient histogram 계산

![24](img/24.png)

- 추가적으로, 특징점 주변의 모든 픽셀들에 대하여 동일하게 생각하지 않고, 특징점과 더 가까이 있는 픽셀들이 중요할테니 특징점을 중심으로 가중치 window를 적용하여 중심에 있는 픽셀들의 가중치를 더 크게 주어서 히스토그램을 계산한다.
    - 이는 위치 강건함(robustness)를 추가하고 외부 영역의 영향을 줄이기 위함이다. </br></br>

> 결과적으로 8x16 값들을 128차원 벡터로 도출된다. </br>
- 조도/광도 불변: </br>
    - 그래디언트 공간을 사용하기 때문에 / = / + b (평행이동)에 강인함
    - 벡터를 [0...1]로 정규화 : / = a/ 밝기 변화에 강인함
    - 0.2보다 큰 벡터 값들은 모두 0.2로 제한(clamping)함
        - "non-linear illumination effects"에 강건함
        - Image value saturation / specular highlights(반사 성분)
    - 재정규화 </br></br>

- 대부분의 feature 기술자는 템플릿, 히스토그램 또는 조합으로 구성됨
- 이상적인 기술자는
    - 강인하고 구별가능함
    - 간결하고 효율적임

- 유용한 대부분의 기술자는 edge / gradient 정보에 중점을 둠
    - 무늬/질감 정보 적용
    - 색상은 거의 사용되지 않음
***

### 매칭 Feature Matching
> 특징 벡터 간의 거리를 계산하여 대응 관계를 수립한다.

- 어떤 특징점이 정합match되는지 어떻게 결정할까?
- 거리 계산의 방법 : Euclidean distance vs Cosine similarity

**Feature Matching**
- 기준 1:
    - 특징점 기술자feature 공간에서 거리를 계산함 -> ex) 128차원 SIFT기술자 사이의 Euclidean distance
    - 가장 가까운 거리를 가진 것들끼리 매칭 </br></br>

- Problems:
    - 모든 특징점을 매칭해야 할까? : 한 영상에서 뽑힌 특징점이 다른 영상에서는 아예 존재하지 않을 수도 있다. 그러므로, 모든 특징점들을 매칭한다기보다 어떤 기준을 넘는 특징점들을 매칭하는게 좋을 것이다!

***

- 기준 2:
    - 특징점 기술자feature 공간에서 거리를 계산함 -> ex) 128차원 SIFT기술자 사이의 Euclidean distance
    - 가장 가까운 거리를 가진 것들끼리 매칭 (nearest neighbor)
    - 거리가 임계값 이상인 점 무시 </br></br>

- Problems:
    - 임계값을 결정하는 것이 어려움
    - 구별되지 않은 feature는 많은 근접 일치점을 가질 수 있으며, 그 중 하나만 정확함 </br>
***

그렇다면, 어떻게 임계값을 효율적으로 정할 수 있을까?

- 가장 가까운 이웃 거리 비율 (NN ratio)

![25](img/25.png)
![26](img/26.png)

    - 가장 가까운(NN1) feature와 두 번째 가까운(NN2) feature벡터 인접 거리를 비교함.
    - 이 비율에 따라 정렬하면 일치 항목이 신뢰도 순서에 따라 정렬된다!
    - 비율 임계값은 거짓 긍정과 실제 긍정 수 사이의 절충에 의해 정해줄 수 있다! </br></br>

- 효율적인 계산 비용
    - 고전적 방법으로 반복하는 것은 계산 비용이 많이 든다.
    - 기술자 행렬에 대해 연산해준다 -> 모든 feature쌍에 대한 내적 결과의 행렬 생성



***
### How good is SIFT?
**SIFT Repeatability**


***
참고문헌 : [컴퓨터비전] 강의6:InterestPoints,
https://angeloyeo.github.io/2020/06/17/Hessian.html
